In [19]:
#importing all libraries
import os
import wandb
import torch
import torch.nn as nn
import pytorch_lightning as pl
from torchvision.models import resnet50
from torchvision import transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
from pytorch_lightning.loggers import WandbLogger
from torchvision.models import ResNet50_Weights
from pytorch_lightning.callbacks import TQDMProgressBar

In [20]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("wandb_key")

os.environ['WANDB_API_KEY'] = secret_value_0

In [21]:
# Freeze all layers in the model
def freeze_all(model):
    for param in model.parameters():
        param.requires_grad = False

# Unfreeze last `k` parameters of the model
def unfreeze_last_k(model, k):
    count = 0
    for name, param in reversed(list(model.named_parameters())):
        if count < k:
            param.requires_grad = True
            count += 1
        else:
            param.requires_grad = False

In [22]:
# Prepare train and validation data loaders
def get_data_loaders(batch_size):
    transform_train = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
    ])

    transform_val = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
    ])

    dataset = ImageFolder("/kaggle/input/inatural-12k/inaturalist_12K/train", transform=transform_train)
    train_len = int(0.8 * len(dataset))
    val_len = len(dataset) - train_len
    train_data, val_data = random_split(dataset, [train_len, val_len])
    val_data.dataset.transform = transform_val

    num_workers = min(4, os.cpu_count()) if torch.cuda.is_available() else 0
    pin_memory = torch.cuda.is_available()

    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True,
                              num_workers=num_workers, pin_memory=pin_memory)
    val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False,
                            num_workers=num_workers, pin_memory=pin_memory)

    return train_loader, val_loader

In [43]:
# LightningModule for ResNet50 fine-tuning
class ResNet50Finetuner(pl.LightningModule):
    def __init__(self, learning_rate, dropout, k):
        super().__init__()
        self.save_hyperparameters()
        self.model = resnet50(weights=ResNet50_Weights.DEFAULT) #pretrained weights

        # Modify the final fully connected layer
        self.model.fc = nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(self.model.fc.in_features, 10)
        )
        # Freeze all layers and unfreeze last k layers
        freeze_all(self.model)
        unfreeze_last_k(self.model, k)
        
        self.criterion = nn.CrossEntropyLoss()
        self.test_accuracy=[]
        self.test_loss=[]

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        inputs, labels = batch
        outputs = self(inputs)
        loss = self.criterion(outputs, labels)
        acc = (outputs.argmax(dim=1) == labels).float().mean()
        self.log("train_loss", loss, prog_bar=True)
        self.log("train_accuracy", acc, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        inputs, labels = batch
        outputs = self(inputs)
        loss = self.criterion(outputs, labels)
        acc = (outputs.argmax(dim=1) == labels).float().mean()
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_accuracy", acc, prog_bar=True)

    def test_step(self, batch, batch_idx):
        inputs, labels = batch
        outputs = self(inputs)
        loss = self.criterion(outputs, labels)
        acc = (outputs.argmax(dim=1) == labels).float().mean()
        self.test_accuracy.append(acc.item())
        self.test_loss.append(loss.item())
        self.log("test_loss", loss, prog_bar=True)
        self.log("test_accuracy", acc, prog_bar=True)

    def on_test_epoch_end(self):
        avg_accuracy = sum(self.test_accuracy) / len(self.test_accuracy)
        avg_loss = sum(self.test_loss) / len(self.test_loss)
        print(f"\nFinal Test Accuracy: {avg_accuracy:.4f}")
        print(f"Final Test Loss: {avg_loss:.4f}")

    def configure_optimizers(self):
        return torch.optim.Adam(
            filter(lambda p: p.requires_grad, self.model.parameters()),
            lr=self.hparams.learning_rate
        )

In [37]:
sweep_config = {
    'method': 'bayes',
    'metric': {
        'name': 'val_accuracy',
        'goal': 'maximize'
    },
    'parameters': {
        'epochs': {'values': [10, 15, 20]},
        'batch_size': {'values': [16, 32, 64]},
        'learning_rate': {'values': [1e-3, 3e-4, 1e-4]},
        'dropout': {'values': [0.2, 0.3, 0.5]},
        'k': {'values': [5, 10, 20, 30]}
    }
}

In [38]:
# Train the model using PyTorch Lightning and wandb
def train():
    run = wandb.init(project="da6401_Assignment2_partB")
    config = wandb.config

    # Construct dynamic run name
    run_name = f"bs={config.batch_size}_lr={config.learning_rate}_k={config.k}_do={config.dropout}"
    run.name = run_name

    model = ResNet50Finetuner(learning_rate=config.learning_rate,dropout=config.dropout,k=config.k)

    train_loader, val_loader = get_data_loaders(config.batch_size)

    wandb_logger = WandbLogger(experiment=run)

    trainer = pl.Trainer(
        max_epochs=config.epochs,
        logger=wandb_logger,
        accelerator="gpu" if torch.cuda.is_available() else "cpu",
        devices=1
    )

    trainer.fit(model, train_loader, val_loader)
    run.finish()

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="da6401_Assignment2_partB")
wandb.agent(sweep_id, function=train, count=30)

**Running on Test data**

In [29]:
best_config = {
    "batch_size": 64,
    "learning_rate": 0.0001,
    "dropout": 0.2,
    "epochs": 10,
    "k": 20
}

In [39]:
# Get test loader
def get_test_loader(batch_size):
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])])
    test_dataset = ImageFolder("/kaggle/input/inatural-12k/inaturalist_12K/val", transform=transform)

    num_workers = min(os.cpu_count(), 4) if torch.cuda.is_available() else 0
    pin_memory = torch.cuda.is_available()

    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False,num_workers=num_workers, pin_memory=pin_memory)
    return test_loader

In [45]:
def trainBestModel():
    model = ResNet50Finetuner(learning_rate=best_config['learning_rate'],dropout=best_config['dropout'],k=best_config['k'])

    train_loader, val_loader = get_data_loaders(best_config['batch_size'])
    
    trainer = pl.Trainer(max_epochs=best_config['epochs'],
        accelerator="gpu" if torch.cuda.is_available() else "cpu",
        devices=1,
        enable_progress_bar=True,
        callbacks=[TQDMProgressBar(refresh_rate=1)])
    
    trainer.fit(model, train_loader, val_loader)
    
    test_loader = get_test_loader(best_config['batch_size'])
    trainer.test(model, test_loader)

In [48]:
trainBestModel()

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Testing: |          | 0/? [00:00<?, ?it/s]


Final Test Accuracy: 0.8516
Final Test Loss: 0.5764


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.8569999933242798     │
│         test_loss         │    0.5544641017913818     │
└───────────────────────────┴───────────────────────────┘